In [1]:
# # # # Web scraping: Africa in International Math Olympiads # # # #

In [3]:
# Help here: https://realpython.com/beautiful-soup-web-scraper-python/

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
URL = 'https://www.imo-official.org/results.aspx'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find(id='main')

In [212]:
countries = results.find_all('tr')
countries.pop(0)
countries.pop(0)

year = [results.find_all('tr')[0]('th')[i]('a')[0]['title'][4:] for i in range(1,len(results.find_all('tr')[0]('th'))-1)]
code = [0 for i in range(len(countries))]
name = [0 for i in range(len(countries))]
data = [[0 for y in year] for i in code]

for i in range(len(countries)):
    code[i] = countries[i].find('a').text
    name[i] = countries[i].find('a')['title']

In [216]:
for i in range(len(countries)-1):
    positions = countries[i].find_all('a')[1:(len(countries[i].find_all('a')))]
    for y in range(len(positions)-1):
        if "team_r.aspx?" in countries[i].find_all('a')[y]["href"]:
            data[i][y] = countries[i].find_all('a')[y].text
            #print(countries[i].find_all('a')[y].text)
#print(code)
#print()